In [2]:
import cv2
import numpy as np
import os
import glob
import json

# Use the same rotate_image_and_box and flip_image_and_box functions from above

# Define a function to perform the augmentation
def augment_dataset(image_dir, label_dir):
    # Get a list of all .png files in the image directory
    image_files = glob.glob(os.path.join(image_dir, "*.png"))

    # For each image file
    for image_file in image_files:
        # Load the image
        image = cv2.imread(image_file)

        # Load the corresponding label
        label_file = os.path.join(label_dir, os.path.basename(image_file).replace(".png", ".txt"))
        with open(label_file, "r") as file:
            label = json.load(file)

        # For each bounding box in the label
        for box in label["features"]:
            # Get the bounding box coordinates
            box_coords = box["properties"]["object_imcoords"].split(",")
            box_coords = list(map(float, box_coords))

            # Rotate the image and bounding box
            angle = np.random.uniform(-180, 180)
            rotated_image, rotated_box = rotate_image_and_box(image, box_coords, angle)

            # Save the rotated images and labels
            cv2.imwrite(os.path.join(image_dir, "augmented", "rotated_" + os.path.basename(image_file)), rotated_image)
            box["properties"]["object_imcoords"] = ",".join(map(str, rotated_box.flatten()))
            with open(os.path.join(label_dir, "augmented", "rotated_" + os.path.basename(label_file)), "w") as file:
                json.dump(label, file)

    # Now flip all the original and rotated images
    all_images = glob.glob(os.path.join(image_dir, "*.png")) + glob.glob(os.path.join(image_dir, "augmented", "*.png"))

    # For each image file
    for image_file in all_images:
        # Load the image
        image = cv2.imread(image_file)

        # Load the corresponding label
        label_file = os.path.join(label_dir, os.path.basename(image_file).replace(".png", ".txt"))
        if "augmented" in image_file:
            label_file = os.path.join(label_dir, "augmented", os.path.basename(image_file).replace(".png", ".txt"))
        with open(label_file, "r") as file:
            label = json.load(file)

        # For each bounding box in the label
        for box in label["features"]:
            # Get the bounding box coordinates
            box_coords = box["properties"]["object_imcoords"].split(",")
            box_coords = list(map(float, box_coords))

            # Flip the image and bounding box
            flipped_image, flipped_box = flip_image_and_box(image, box_coords)

            # Save the flipped images and labels
            cv2.imwrite(os.path.join(image_dir, "augmented", "flipped_" + os.path.basename(image_file)), flipped_image)
            box["properties"]["object_imcoords"] = ",".join(map(str, flipped_box))
            with open(os.path.join(label_dir, "augmented", "flipped_" + os.path.basename(label_file)), "w") as file:
                json.dump(label, file)

# Use the function
augment_dataset("train_images/", "train_labels_dota/")


ModuleNotFoundError: No module named 'cv2'